# 🚀 1. Pydantic
###### Al momento de hacer este trabajo, la documentación de pydantic se encuentra en la siguiente dirección: https://pydantic-docs.helpmanual.io/
Pydantic es una biblioteca de Python que permite validar datos y gestionar configuraciones de manera sencilla y potente. Es especialmente útil en aplicaciones web, APIs y para cualquier situación donde necesitemos validar estructuras de datos.

⚡ Para comenzar, no olvides generar un entorno virtual e instalar pydantic con pip:
```bash
pip install pydantic
```
Luego, se irán importantdo las clases `BaseModel` y todas aquellas que se vayan necesitando.       

***Pydantic*** es una biblioteca para validación de datos basada en anotaciones de tipo de Python. Permite definir la estructura, los tipos y las validaciones de tus datos de forma clara y concisa. Una de sus principales ventajas es que aprovecha el sistema de tipado estático de Python, lo que facilita la detección de errores en tiempo de desarrollo.


In [2]:
# 1. INTRODUCCIÓN A PYDANTIC
from pydantic import BaseModel, Field, ValidationError, field_validator, model_validator
from typing import List, Optional, Dict, Union
from datetime import datetime

## 📌 2. Módelos Básicos
Para definir un modelo en Pydantic, simplemente creamos una clase que herede de `BaseModel` y definimos los campos que queremos validar. 
- Por ejemplo, si queremos validar un modelo de usuario con un nombre y una edad, podemos hacerlo de la siguiente manera:

In [3]:
# 2. MODELO BÁSICO
class Usuario(BaseModel):
    id: int
    nombre: str
    apellido: str
    edad: int
    email: str
    activo: bool = True  # Campo con valor por defecto

# Crear instancia
usuario = Usuario(
    id=1,
    nombre="Juan",
    apellido="Pérez",
    edad=30,
    email="juan.perez@ejemplo.com"
)

print(f"Usuario creado: {usuario.model_dump()}")

Usuario creado: {'id': 1, 'nombre': 'Juan', 'apellido': 'Pérez', 'edad': 30, 'email': 'juan.perez@ejemplo.com', 'activo': True}


## 📌 3. Validación de tipos
Pydantic nos permite definir los tipos de los campos de nuestros modelos de forma sencilla. 
- Por ejemplo, si queremos que el campo `name` sea de tipo `str` y el campo `age` sea de tipo `int`, simplemente tenemos que añadir la anotación de tipo correspondiente a cada campo.
- Pydantic se encargará de validar que los datos que le pasemos cumplan con los tipos definidos en el modelo.
  - Si los datos **no cumplen** con los tipos definidos, Pydantic lanzará una excepción `ValidationError` con un mensaje de error detallado.
  - Si los datos **cumplen** con los tipos definidos, Pydantic devolverá una instancia del modelo con los datos validados.


In [4]:
# 3. VALIDACIÓN DE DATOS
# Esto causará una ValidationError porque 'email' es requerido
try:
    usuario_invalido = Usuario(
        id=2,
        nombre="Ana",
        apellido="García",
        edad=25
    )
except ValidationError as e:
    print(f"Error de validación: {e}")

Error de validación: 1 validation error for Usuario
email
  Field required [type=missing, input_value={'id': 2, 'nombre': 'Ana'...: 'García', 'edad': 25}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing


## 📌 4. Tipos Complejos
Pydantic es capaz de manejar estructuras de datos complejas. Podemos definir modelos anidados, listas, diccionarios y otros tipos complejos.
Acá estamos usando tipos como `Optional`, `List`, `Dict` y modelos anidados como `Dirección`.
- Por ejemplo, si queremos validar un modelo de usuario con un nombre, una edad y una dirección, podemos hacerlo de la siguiente manera:

In [5]:
# 4. TIPOS COMPLEJOS
class Dirección(BaseModel):
    calle: str
    ciudad: str
    código_postal: str
    país: str = "España"

class UsuarioDetallado(BaseModel):
    id: int
    nombre: str
    apellido: str
    edad: int
    email: str
    dirección: Optional[Dirección] = None
    etiquetas: List[str] = []
    metadatos: Dict[str, Union[str, int, bool]] = {}
    fecha_registro: datetime = Field(default_factory=datetime.now)
    activo: bool = True
    
# Crear usuario con tipos complejos
usuario_detallado = UsuarioDetallado(
    id=3,
    nombre="Carlos",
    apellido="Rodríguez",
    edad=35,
    email="carlos@ejemplo.com",
    dirección={
        "calle": "Calle Mayor 123",
        "ciudad": "Madrid",
        "código_postal": "28001"
    },
    etiquetas=["premium", "desarrollador"],
    metadatos={"plan": "pro", "días_activo": 120, "descuentos": True}
)

print(f"\nUsuario detallado: {usuario_detallado.model_dump()}")
print(f"Fecha de registro: {usuario_detallado.fecha_registro}")


Usuario detallado: {'id': 3, 'nombre': 'Carlos', 'apellido': 'Rodríguez', 'edad': 35, 'email': 'carlos@ejemplo.com', 'dirección': {'calle': 'Calle Mayor 123', 'ciudad': 'Madrid', 'código_postal': '28001', 'país': 'España'}, 'etiquetas': ['premium', 'desarrollador'], 'metadatos': {'plan': 'pro', 'días_activo': 120, 'descuentos': True}, 'fecha_registro': datetime.datetime(2025, 3, 3, 8, 49, 31, 823321), 'activo': True}
Fecha de registro: 2025-03-03 08:49:31.823321


## 📌 5. Validadores Personalizados
Frecuentemente necesitamos reglas de validación específicas. 
- Pydantic nos permite crear validadores personalizados a nivel de campo y a nivel de modelo completo.

In [6]:
# 5. VALIDADORES DE CAMPO
class ProductoModel(BaseModel):
    id: int
    nombre: str
    precio: float
    stock: int
    
    @field_validator('precio')
    @classmethod
    def validar_precio(cls, v):
        if v <= 0:
            raise ValueError('El precio debe ser mayor que cero')
        return round(v, 2)  # Redondear a 2 decimales
    
    @field_validator('stock')
    @classmethod
    def validar_stock(cls, v):
        if v < 0:
            raise ValueError('El stock no puede ser negativo')
        return v

try:
    producto_valido = ProductoModel(id=1, nombre="Laptop", precio=1299.99, stock=10)
    print(f"\nProducto válido: {producto_valido.model_dump()}")
    
    producto_invalido = ProductoModel(id=2, nombre="Smartphone", precio=-500, stock=5)
except ValidationError as e:
    print(f"\nError de validación en producto: {e}")

# 6. VALIDADORES DE MODELO
class Pedido(BaseModel):
    id: int
    producto_id: int
    cantidad: int
    precio_unitario: float
    precio_total: float = 0
    
    @model_validator(mode='after')
    def calcular_precio_total(self):
        self.precio_total = round(self.cantidad * self.precio_unitario, 2)
        return self

pedido = Pedido(id=1, producto_id=1, cantidad=3, precio_unitario=1299.99)
print(f"\nPedido con precio total calculado: {pedido.model_dump()}")


Producto válido: {'id': 1, 'nombre': 'Laptop', 'precio': 1299.99, 'stock': 10}

Error de validación en producto: 1 validation error for ProductoModel
precio
  Value error, El precio debe ser mayor que cero [type=value_error, input_value=-500, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

Pedido con precio total calculado: {'id': 1, 'producto_id': 1, 'cantidad': 3, 'precio_unitario': 1299.99, 'precio_total': 3899.97}


## 📌 6. Serialización y Deserialización
Pydantic facilita la conversión entre objetos Python y formatos como JSON, lo que es esencial para APIs.
- Podemos convertir nuestros modelos a JSON y viceversa de forma muy sencilla.

In [7]:
# 7. SERIALIZACIÓN (JSON)
usuario_json = usuario_detallado.model_dump_json()
print(f"\nUsuario en JSON:\n{usuario_json}")

# 8. DESERIALIZACIÓN
datos_json = '''
{
    "id": 4,
    "nombre": "Laura",
    "apellido": "Martínez",
    "edad": 28,
    "email": "laura@ejemplo.com",
    "dirección": {
        "calle": "Avenida Libertad 456",
        "ciudad": "Barcelona",
        "código_postal": "08001"
    }
}
'''

usuario_desde_json = UsuarioDetallado.model_validate_json(datos_json)
print(f"\nUsuario creado desde JSON: {usuario_desde_json.model_dump()}")

# 9. CONFIG Y ALIAS
class ConfigDemo(BaseModel):
    nombre_usuario: str = Field(alias="userName")
    contraseña: str = Field(alias="password")
    
    model_config = {
        "populate_by_name": True,
        "str_strip_whitespace": True,
        "validate_assignment": True
    }

config_demo = ConfigDemo(userName="admin", contraseña="segura123")
print(f"\nConfig demo: {config_demo.model_dump()}")
print(f"Acceso por alias: {config_demo.nombre_usuario}")


Usuario en JSON:
{"id":3,"nombre":"Carlos","apellido":"Rodríguez","edad":35,"email":"carlos@ejemplo.com","dirección":{"calle":"Calle Mayor 123","ciudad":"Madrid","código_postal":"28001","país":"España"},"etiquetas":["premium","desarrollador"],"metadatos":{"plan":"pro","días_activo":120,"descuentos":true},"fecha_registro":"2025-03-03T08:49:31.823321","activo":true}

Usuario creado desde JSON: {'id': 4, 'nombre': 'Laura', 'apellido': 'Martínez', 'edad': 28, 'email': 'laura@ejemplo.com', 'dirección': {'calle': 'Avenida Libertad 456', 'ciudad': 'Barcelona', 'código_postal': '08001', 'país': 'España'}, 'etiquetas': [], 'metadatos': {}, 'fecha_registro': datetime.datetime(2025, 3, 3, 8, 49, 31, 854389), 'activo': True}

Config demo: {'nombre_usuario': 'admin', 'contraseña': 'segura123'}
Acceso por alias: admin


## 📌 7. Configuración y Alias
Pydantic permite personalizar el comportamiento de nuestros modelos y usar alias para nuestros campos.
- Podemos configurar nuestros modelos para que ignoren campos desconocidos, para que conviertan automáticamente los nombres de los campos a snake_case, para que conviertan automáticamente los valores de los campos a mayúsculas, etc.
- También podemos usar alias para que nuestros campos tengan nombres diferentes en Python y en JSON.  
-Esto es especialmente útil cuando trabajamos con APIs externas que usan convenciones de nombres diferentes.

In [8]:
# 9. CONFIG Y ALIAS
class ConfigDemo(BaseModel):
    nombre_usuario: str = Field(alias="userName")
    contraseña: str = Field(alias="password")
    
    model_config = {
        "populate_by_name": True,
        "str_strip_whitespace": True,
        "validate_assignment": True
    }
config_demo = ConfigDemo(userName="admin", contraseña="segura123")
print(f"\nConfig demo: {config_demo.model_dump()}")
print(f"Acceso por alias: {config_demo.nombre_usuario}")


Config demo: {'nombre_usuario': 'admin', 'contraseña': 'segura123'}
Acceso por alias: admin


## 8. 📌 Herencia de Modelos
Podemos aprovechar la herencia para reutilizar campos comunes en diferentes modelos.
- Esto nos ayuda a mantener nuestro código DRY (Don't Repeat Yourself).

In [9]:
# 10. HERENCIA DE MODELOS
class ModeloBase(BaseModel):
    id: int
    fecha_creación: datetime = Field(default_factory=datetime.now)

class Producto(ModeloBase):
    nombre: str
    precio: float

class Cliente(ModeloBase):
    nombre: str
    email: str

producto = Producto(id=1, nombre="Monitor", precio=249.99)
cliente = Cliente(id=1, nombre="Pedro", email="pedro@ejemplo.com")

print(f"\nProducto heredado: {producto.model_dump()}")
print(f"Cliente heredado: {cliente.model_dump()}")


Producto heredado: {'id': 1, 'fecha_creación': datetime.datetime(2025, 3, 3, 8, 49, 31, 878643), 'nombre': 'Monitor', 'precio': 249.99}
Cliente heredado: {'id': 1, 'fecha_creación': datetime.datetime(2025, 3, 3, 8, 49, 31, 878643), 'nombre': 'Pedro', 'email': 'pedro@ejemplo.com'}


## 9. 📌 Casos de Uso Reales: Schemas para APIs
Pydantic es ampliamente utilizado en frameworks como FastAPI. 
- Vamos a ver un ejemplo de cómo usar Pydantic para definir schemas de API."
- Típicamente tendríamos schemas separados para creación, actualización y respuestas de API.

In [10]:
# 📌 11. SCHEMAS PARA API (PARA ILUSTRAR CASOS DE USO REALES)
class ProductoCrear(BaseModel):
    nombre: str
    precio: float
    stock: int

class ProductoActualizar(BaseModel):
    nombre: Optional[str] = None
    precio: Optional[float] = None
    stock: Optional[int] = None

class ProductoRespuesta(ModeloBase):
    nombre: str
    precio: float
    stock: int

# Simular API
def crear_producto(datos: ProductoCrear) -> ProductoRespuesta:
    # Aquí iría lógica para guardar en base de datos
    return ProductoRespuesta(
        id=100,
        nombre=datos.nombre,
        precio=datos.precio,
        stock=datos.stock
    )

datos_producto = ProductoCrear(nombre="Teclado", precio=89.99, stock=20)
producto_creado = crear_producto(datos_producto)
print(f"\nProducto creado en API: {producto_creado.model_dump()}")


Producto creado en API: {'id': 100, 'fecha_creación': datetime.datetime(2025, 3, 3, 8, 49, 31, 890984), 'nombre': 'Teclado', 'precio': 89.99, 'stock': 20}


Autor: Daniel Christello
_____________________________________________________________